In [1]:
# A/B Testing is a process of conducting an experiment and testing the hypothesis between
# control and treatment groups.

# Steps involved are - Designing Experiment, Collecting Data including Sample size,
# EDA, Testing Hypothesis, Infer the test results.

# Example Designed a new typeof product page and are trying to find out whether the new page
# or old page is effective in bringing leads to business.

# Formulate a Hypothesis and will conduct a 2 tail hypothesis test between the new page and old page lead generation rate
# Po represents old page and Pn represents new page

# Null - Po = Pn

# Alt - Po!=Pn

# Confidence level - 95% (0.05 Alpha)

# Choosing Variables - 2 Groups are needed
# Control group - They will be sown only Old page
# Treatment group(Experimental Group) - They will be shown oly New Page
# Dependent Variable - Lead_Generted - Yes/No

# Choosing the Right Sample Size - Using a method Power Analysis 
# Power of Test (1-beta) - probability of finding a stattiscial difference between the groups 
# when the differences is actually present. Usually it is set to 0.80.
# alpha = 0.05

# Effect Size - How big differences is expected between the groups lead generation (percentage)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
abdf = pd.read_csv('ab_data.csv')

In [6]:
abdf.shape

(294478, 5)

In [4]:
abdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [7]:
abdf.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [8]:
abdf.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [10]:
# Calculating Required Sample Size

from statsmodels.stats.api import NormalIndPower,proportion_effectsize
effct_size = proportion_effectsize(0.13, 0.15) # Estimated Effect Size(13-15%)
required_sample = NormalIndPower().solve_power(effect_size=effct_size, power=0.80, alpha=0.05,ratio=1)

In [12]:
required_sample = np.ceil(required_sample)

In [13]:
sess_counts = abdf.user_id.value_counts(ascending=False)

In [14]:
duplicate = sess_counts[sess_counts>1].count()

In [15]:
duplicate

3894

In [16]:
duplicate_users = sess_counts[sess_counts>1].index

In [17]:
abdf = abdf[~abdf.user_id.isin(duplicate_users)]

In [18]:
abdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286690 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       286690 non-null  int64 
 1   timestamp     286690 non-null  object
 2   group         286690 non-null  object
 3   landing_page  286690 non-null  object
 4   converted     286690 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.1+ MB


In [19]:
control_sample = abdf[abdf.group=="control"].sample(n=required_sample.astype(int),random_state=42)

In [20]:
experimental_sample = abdf[abdf.group=="treatment"].sample(n=required_sample.astype(int),random_state=42)

In [21]:
ab_test = pd.concat([control_sample, experimental_sample],axis = 0)

In [22]:
ab_test.reset_index(drop=True,inplace=True)

In [23]:
ab_test.group.value_counts()

control      4720
treatment    4720
Name: group, dtype: int64

In [28]:
control_results = ab_test[ab_test.group=='control']['converted']

In [30]:
experimental_results = ab_test[ab_test.group=='treatment']['converted']

In [31]:
from statsmodels.stats.proportion import proportions_ztest

In [32]:
success = [control_results.sum(),experimental_results.sum()]

In [33]:
nobs = [control_results.count(), experimental_results.count()]

In [34]:
z_stat,pval = proportions_ztest(success,nobs)

In [35]:
z_stat

-2.0109005932696107

In [36]:
pval #Since p-value is less than 0.05, Reject null

0.044335957690484505